# Basic Planning


* We have roughly ~11k images with faces detected using the YOLO model. Now, the goal of this is to filter the dataset so that we can get a few faces with decent base resolution for the face generation training.
* Since we are generating the face to be around 128x128 pixels, we can see how many faces directly fall into this category. If the number is insufficient (or) small, then we can check for the faces larger than 64x64 pixels and then resize them to 128x128 pixels.


In [2]:
import os
from PIL import Image
from tqdm import tqdm


image_dir = "test_images"
bbox_dir = "inference_results" #inference results from YOLO

def filter_128_faces(image_name):
    image_path = os.path.join(image_dir, image_name)
    bbox_path = os.path.join(bbox_dir, image_name.replace('.jpg', '.txt'))

    if not os.path.exists(bbox_path):
        return False

    with open(bbox_path, 'r') as f:
        lines = f.readlines()

    os.makedirs("filtered_faces_100", exist_ok=True)
    for line in lines:
        parts = line.strip().split()
        if len(parts) < 5:
            continue
        x1, y1, w, h = map(float, parts[2:6])
        x2 = x1 + w
        y2 = y1 + h
        
        width = x2 - x1
        height = y2 - y1

        if width*height >= 64*64:
            image = Image.open(image_path)
            face = image.crop((x1, y1, x2, y2))
            face = face.resize((128, 128))
            face.save(os.path.join("filtered_faces_100", image_name))
            return True

    return False

images = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

remaining_images=[]

for img in tqdm(images):
    filtered = filter_128_faces(img)
    if not filtered:
        remaining_images+=[img]

number_of_faces = len(os.listdir("filtered_faces_100"))
print(f"Number of faces with size >= 64x64: {number_of_faces}")

100%|██████████| 11316/11316 [00:22<00:00, 493.46it/s]

Number of faces with size >= 128x128: 7285


In [3]:
from sklearn.model_selection import train_test_split
images = os.listdir("filtered_faces_100")
train_images, test_images = train_test_split(images, test_size=0.1, random_state=42)
train_images, val_images = train_test_split(train_images, test_size=0.15, random_state=42)

os.makedirs("data/train", exist_ok=True)
os.makedirs("data/val", exist_ok=True)
os.makedirs("data/test", exist_ok=True)

import shutil
for img in train_images:
    shutil.copy(os.path.join("filtered_faces_100", img), os.path.join("data/train", img))
for img in val_images:
    shutil.copy(os.path.join("filtered_faces_100", img), os.path.join("data/val", img))
for img in test_images:
    shutil.copy(os.path.join("filtered_faces_100", img), os.path.join("data/test", img))